In [13]:
import re
import numpy as np
import pandas as pd

In [14]:
annot = pd.read_csv('/home/lucytian/data/1_Single_Cell_PRS/2_cV2F/ukb_genoHM3_mvp_eur_tissue_cV2F.annot.pvar.gz', sep='\t', compression='gzip')

/tmp/ipykernel_306209/3794331728.py:1: DtypeWarning: Columns (0,9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  annot = pd.read_csv('/home/lucytian/data/1_Single_Cell_PRS/2_cV2F/ukb_genoHM3_mvp_eur_tissue_cV2F.annot.pvar.gz', sep='\t', compression='gzip')


In [15]:
N_dict = {'Afr': 4853, 'WB': 270920}
sumstat_dir = '/home/lucytian/data/4_polypred/gwas_geno'


for pop in ['Afr', 'WB']:
    if pop == 'Afr':
        paths = {
            'INI30120': f"{sumstat_dir}/blood_count/pop_10PCs/{pop}_train/INI30120.glm.tsv.gz",   # lymphocytes
            'INI20030780': f"{sumstat_dir}/blood_biochemistry/pop_10PCs/{pop}_train/INI20030780.glm.tsv.gz",  # LDL-C
            'INI1003063': f"{sumstat_dir}/spirometry/pop_10PCs/{pop}_train/INI1003063.glm.tsv.gz",            # FEV1/FVC
            'INI50030700': f"{sumstat_dir}/blood_biochemistry/pop_10PCs/{pop}_train/INI50030700.glm.tsv.gz",  # eGFR
        }
    else:
        paths = {
            'INI30120': f"{sumstat_dir}/blood_count/pop_10PCs/{pop}_train_val/INI30120.glm.tsv.gz",   # lymphocytes
            'INI20030780': f"{sumstat_dir}/blood_biochemistry/pop_10PCs/{pop}_train_val/INI20030780.glm.tsv.gz",  # LDL-C
            'INI1003063': f"{sumstat_dir}/spirometry/pop_10PCs/{pop}_train_val/INI1003063.glm.tsv.gz",            # FEV1/FVC
            'INI50030700': f"{sumstat_dir}/blood_biochemistry/pop_10PCs/{pop}_train_val/INI50030700.glm.tsv.gz",  # eGFR
        }

    for trait_id, p in paths.items():
        print(trait_id)

        df = pd.read_csv(p, compression='gzip', sep='\t')

        # Basic column checks (adjust names if your headers differ)
        needed = {'ID','A1','REF','ALT','ALT_FREQS','BETA','SE'}
        missing = needed - set(df.columns)
        if missing:
            raise ValueError(f"{trait_id}: Missing columns in {p}: {missing}")

        # p-value: your file has LOG10_P; convert to raw p
        if 'LOG10_P' in df.columns and 'P' not in df.columns:
            df['p'] = 10.0 ** (-df['LOG10_P'].astype(float))
        elif 'P' in df.columns:
            df['p'] = df['P'].astype(float)
        else:
            raise ValueError(f"{trait_id}: Need LOG10_P or P in {p}")

        # Uppercase alleles
        for col in ['A1','REF','ALT']:
            df[col] = df[col].astype(str).str.upper()
            
            
        merged = df.merge(annot[['ID','rsID']], on='ID')   
        

        # Effect-allele frequency: A1 may be ALT or REF; ALT_FREQS is freq(ALT)
        is_a1_alt = (merged['A1'] == merged['ALT'])
        freq = np.where(is_a1_alt, merged['ALT_FREQS'].astype(float), 1.0 - merged['ALT_FREQS'].astype(float))

        # Non-effect allele (A2) = the other allele
        A2 = np.where(is_a1_alt, merged['REF'], merged['ALT'])
        

        # Assemble COJO .ma
        out = pd.DataFrame({
            'SNP':  merged['rsID'],
            'A1':   merged['A1'],
            'A2':   A2,
            'freq': freq,
            'b':    merged['BETA'].astype(float),
            'se':   merged['SE'].astype(float),
            'p':    merged['p'].astype(float).clip(lower=1e-300, upper=1.0),
            'N':    N_dict[pop]
        })

        # Drop any incomplete rows
        out = out.dropna(subset=['SNP','A1','A2','freq','b','se','p','N'])
        

        # (Optional) remove strand-ambiguous with near-0.5 freq if you want:
        # amb = ( (out['A1']+out['A2']).isin(['AT','TA','CG','GC']) ) & (np.isclose(out['freq'], 0.5, atol=0.02))
        # out = out.loc[~amb]

        out_path = f'input_sumstat/{pop}_{trait_id}.ma'
        out.to_csv(out_path, sep='\t', index=False)
        print(f"wrote {out_path}  (n={len(out):,})")


INI30120


/tmp/ipykernel_306209/489944072.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


wrote input_sumstat/Afr_INI30120.ma  (n=1,255,370)
INI20030780


/tmp/ipykernel_306209/489944072.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


wrote input_sumstat/Afr_INI20030780.ma  (n=1,255,519)
INI1003063


...

wrote input_sumstat/WB_INI20030780.ma  (n=1,313,801)
INI1003063


/tmp/ipykernel_306209/489944072.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


wrote input_sumstat/WB_INI1003063.ma  (n=1,313,743)
INI50030700


/tmp/ipykernel_306209/489944072.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


wrote input_sumstat/WB_INI50030700.ma  (n=1,313,802)
